In [ ]:
%env  WORKDIR=/tmp/vault

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

## Step 1. Install MongoDB

In [ ]:
! kubectl apply -f mongodb_deploy.yaml

In [ ]:
! kubectl get pods -n mongodb

# Step 2. Configure Vault

### Enable Secret Engine

### We are using the previously created engine. But if you want you can create another engine on a different path

### Create Connection

In [ ]:
%%bash
export MONGO_ADDR=mongodb.mongodb.svc.cluster.local
export USERNAME="mdbadmin"
export PASSWORD="Passw0rd123!"

vault write database/config/mongodb \
    plugin_name=mongodb-database-plugin \
    allowed_roles="*"  connection_url="mongodb://{{username}}:{{password}}@${MONGO_ADDR}:27017/admin?tls=false" \
     username=$USERNAME \
     password=$PASSWORD

Rotate root

In [ ]:
! vault write -force database/rotate-root/mongodb

### Create Role

In [ ]:
%%bash

vault write database/roles/mongo-test \
    db_name=mongodb \
    creation_statements='{ "db": "admin", "roles": [{ "role": "readWrite" }, {"role": "read", "db": "foo"}] }' \
    default_ttl="1h" \
    max_ttl="24h"

In [ ]:
%%bash
vault write database/roles/mongo-admin \
    db_name=mongodb \
    creation_statements='{
      "db": "admin",
      "roles": [
        { "role": "root", "db": "admin" }
      ]
    }' \
    default_ttl="1h" \
    max_ttl="24h"

### Create Dynamic Creds via CLI

In [ ]:
! vault read database/creds/mongo-test

In [ ]:
! vault read database/creds/mongo-admin

# Clean UP